In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
MODEL_PATH = "/kaggle/input/llama-3.1/transformers/8b-instruct/2"
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/llama-3.1/transformers/8b-instruct/2/model.safetensors.index.json
/kaggle/input/llama-3.1/transformers/8b-instruct/2/model-00003-of-00004.safetensors
/kaggle/input/llama-3.1/transformers/8b-instruct/2/config.json
/kaggle/input/llama-3.1/transformers/8b-instruct/2/LICENSE
/kaggle/input/llama-3.1/transformers/8b-instruct/2/model-00001-of-00004.safetensors
/kaggle/input/llama-3.1/transformers/8b-instruct/2/README.md
/kaggle/input/llama-3.1/transformers/8b-instruct/2/USE_POLICY.md
/kaggle/input/llama-3.1/transformers/8b-instruct/2/tokenizer.json
/kaggle/input/llama-3.1/transformers/8b-instruct/2/tokenizer_config.json
/kaggle/input/llama-3.1/transformers/8b-instruct/2/model-00004-of-00004.safetensors
/kaggle/input/llama-3.1/transformers/8b-instruct/2/special_tokens_map.json
/kaggle/input/llama-3.1/transformers/8b-instruct/2/.gitattributes
/kaggle/input/llama-3.1/transformers/8b-instruct/2/model-00002-of-00004.safetensors
/kaggle/input/llama-3.1/transformers/8b-instruct/2/gener

In [2]:
!pip install --upgrade unsloth


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.4/57.4 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.6/187.6 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 81.7 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.1/253.1 MB 6.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.3/318.3 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 MB 41.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 2.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.9 MB/s eta 0:00:000:00:0100

In [3]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None
load_in_4bit = True 

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "/kaggle/input/llama-3.1/transformers/8b-instruct/2",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.2.12: Fast Llama patching. Transformers: 4.49.0.
   \\   /|    GPU: Tesla P100-PCIE-16GB. Max memory: 15.888 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 6.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

/kaggle/input/llama-3.1/transformers/8b-instruct/2 does not have a padding token! Will use pad_token = <|finetune_right_pad_id|>.


In [4]:
# Check if GPU is available
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available. Using GPU.")
    gpu_stats = torch.cuda.get_device_properties(0)
    print(f"GPU Name: {gpu_stats.name}")
    max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
    print(f"GPU Max memory = {max_memory} GB.")
else:
    device = torch.device("cpu")
    print("GPU is not available. Using CPU.")

GPU is available. Using GPU.
GPU Name: Tesla P100-PCIE-16GB
GPU Max memory = 15.888 GB.


In [5]:
import numpy as np
import pandas as pd
import os
import torch
from datasets import load_dataset
from transformers import TrainingArguments, DataCollatorForSeq2Seq

# Configuration parameters - making them explicit
MAX_SEQ_LENGTH = 2048
DATA_PATH = "/kaggle/input/multi-lingual-sentiment-analysis/train.csv"
TEST_PATH = "/kaggle/input/multi-lingual-sentiment-analysis/test.csv"
SUBMISSION_PATH = "/kaggle/input/multi-lingual-sentiment-analysis/sample_submission.csv"
OUTPUT_DIR = "fine_tuned_model"
LORA_RANK = 8
LEARNING_RATE = 2e-4
TRAIN_STEPS = 80
BATCH_SIZE = 8
GRADIENT_ACCUMULATION = 2
WARMUP_STEPS = 5
SEED_VALUE = 42

print("Starting the sentiment analysis pipeline...")


#Try importing FastLanguageModel and other unsloth modules within a try-except block
try:
    from unsloth import FastLanguageModel
    from unsloth import is_bfloat16_supported
    from unsloth.chat_templates import standardize_sharegpt
    from unsloth.chat_templates import get_chat_template
    print("Successfully imported Unsloth modules")
except ImportError as e:
    print(f"Error importing Unsloth modules: {e}")
    print("The Unsloth library could not be imported. Please ensure it is installed correctly.")
    raise  # Re-raise the exception to stop the execution if Unsloth is essential



# Model Loading and Configuration
print(f"Loading model: {MODEL_PATH}")
try:
    base_model, tokenizer = FastLanguageModel.from_pretrained(
        model_name=MODEL_PATH,
        max_seq_length=MAX_SEQ_LENGTH,
        dtype=None,
        load_in_4bit=True
    )
except NameError as e:
    print(f"Error loading model: {e}")
    print("This is likely because Unsloth modules were not imported correctly.  The program will terminate.")
    raise


print("Applying LoRA...")
try:
    fine_tuned_model = FastLanguageModel.get_peft_model(
        base_model,
        r=LORA_RANK,  # Reduced LoRA rank slightly
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                        "gate_proj", "up_proj", "down_proj", ],
        lora_alpha=16,
        lora_dropout=0,
        bias="none",
        use_gradient_checkpointing="unsloth",
        random_state=SEED_VALUE,  # Consistent random seed
        use_rslora=False,
        loftq_config=None,
    )
except NameError as e:
    print(f"Error applying LoRA: {e}")
    print("This is likely because Unsloth modules were not imported correctly.  The program will terminate.")
    raise



# Dataset Preparation
print(f"Loading dataset from: {DATA_PATH}")
raw_dataset = load_dataset("csv", data_files=DATA_PATH, split="train")
print(f"Dataset loaded: {raw_dataset}")



def zero_shot_transform(example):
    instruction = "Determine the sentiment of the following sentence.  Answer 'positive' or 'negative'."
    return {
        "conversations": [
            {"from": "human", "value": f"{instruction}\n\n{example['sentence']}"},
            {"from": "gpt", "value": str(example['label'])}
        ],
        "language": example["language"]
    }

print("Transforming dataset...")
processed_dataset = raw_dataset.map(zero_shot_transform, remove_columns=raw_dataset.column_names)
print(f"Transformed dataset example: {processed_dataset[0]}")

print("Standardizing ShareGPT format...")
try:
    standardized_dataset = standardize_sharegpt(processed_dataset)
except NameError as e:
    print(f"Error standardizing ShareGPT format: {e}")
    print("This likely occurred because the Unsloth import failed. Proceeding without standardization. "
          "This may negatively impact performance.")
    standardized_dataset = processed_dataset # Use the non-standardized data as a fallback


def tokenize_prompt(examples):
    conversations = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False) for convo in conversations]
    return {"text": texts}

print("Tokenizing prompts...")
try:
    tokenized_dataset = standardized_dataset.map(tokenize_prompt, batched=True)
except NameError as e:
    print(f"Error tokenizing prompts: {e}")
    print("Tokenizer is not defined. Likely due to a failed import.")
    raise # Exit.  Inference will not be possible without a tokenizer.

# Attempt to import SFTTrainer here, after loading data, in case TensorFlow issue is related to import order
try:
    from trl import SFTTrainer
    from unsloth.chat_templates import train_on_responses_only
    print("Successfully imported SFTTrainer from trl")

    # Trainer Configuration
    print("Configuring trainer...")
    try:
        trainer = SFTTrainer(
            model=fine_tuned_model,
            tokenizer=tokenizer,
            train_dataset=tokenized_dataset,
            dataset_text_field="text",
            max_seq_length=MAX_SEQ_LENGTH,
            data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer),
            dataset_num_proc=2,
            packing=False,
            args=TrainingArguments(
                per_device_train_batch_size=BATCH_SIZE,
                gradient_accumulation_steps=GRADIENT_ACCUMULATION,
                warmup_steps=WARMUP_STEPS,
                max_steps=TRAIN_STEPS,  # Limited training steps
                learning_rate=LEARNING_RATE,
                fp16=not is_bfloat16_supported(),
                bf16=is_bfloat16_supported(),
                logging_steps=1,
                logging_strategy = "steps",       # Logs metrics at each specified step
                optim="adamw_8bit",
                weight_decay=0.01,
                lr_scheduler_type="linear",
                seed=SEED_VALUE,
                output_dir=OUTPUT_DIR,
                report_to="none",
                save_strategy="no",  # Disable checkpoint saving
                load_best_model_at_end = True    # Loads the best model at the end

            ),
        )
    except NameError as e:
        print(f"Error creating trainer: {e}")
        print("This is likely because Unsloth modules were not imported correctly. The program will terminate.")
        raise #Re-raise to stop execution

    print("Training on responses only...")
    try:
        trainer = train_on_responses_only(
            trainer,
            instruction_part="<|start_header_id|>user<|end_header_id|>\n\n",
            response_part="<|start_header_id|>assistant<|end_header_id|>\n\n",
        )
    except NameError as e:
         print(f"Error during train on responses only: {e}")
         print("This is likely because Unsloth modules were not imported correctly.  Training cannot proceed")
         raise # Terminate

    # Training Loop
    print("Starting training...")
    gpu_stats = torch.cuda.get_device_properties(0)
    start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
    max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)

    print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
    print(f"{start_gpu_memory} GB of memory reserved before training.")

    try:
        trainer_stats = trainer.train()
    except NameError as e:
        print(f"Error during trainer.train(): {e}")
        print("Training failed. Check the Unsloth installation and your code.")
        raise #Stop Execution

    # Post-Training Analysis
    used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
    used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
    used_percentage = round(used_memory / max_memory * 100, 3)
    lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)

    print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
    print(f"{round(trainer_stats.metrics['train_runtime'] / 60, 2)} minutes used for training.")
    print(f"Peak reserved memory = {used_memory} GB.")
    print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
    print(f"Peak reserved memory % of max memory = {used_percentage} %.")
    print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

except ImportError as e:
    print(f"Error importing SFTTrainer: {e}")
    print("It's likely there's a conflict with TensorFlow or another dependency.")
    print("Skipping the training phase. Proceeding to inference with the base model.")
    fine_tuned_model = base_model #Revert to using the base model


# Inference Preparation
print("Preparing for inference...")
try:
    tokenizer = get_chat_template(
        tokenizer,
        chat_template="llama-3.1",
    )
    FastLanguageModel.for_inference(fine_tuned_model)
except NameError as e:
    print(f"Error preparing for inference: {e}")
    print("Tokenizer or FastLanguageModel is not defined. Likely due to a failed import.")
    raise # Can't continue.  Tokenizer is essential for inference

def predict_sentiment(model, tokenizer, sentence):
    messages = [{"role": "user", "content": sentence}]
    try:
        inputs = tokenizer.apply_chat_template(
            messages,
            tokenize=True,
            add_generation_prompt=True,
            return_tensors="pt",
        ).to("cuda")
    except NameError as e:
        print(f"Error in predict_sentiment during apply_chat_template: {e}")
        print("Tokenizer is not defined. Likely due to a failed import.")
        return "Error: Tokenizer not available"

    try:
        outputs = model.generate(
            input_ids=inputs,
            max_new_tokens=64,
            use_cache=True,
            temperature=0.1,
            min_p=0.1
        )
        output_text = tokenizer.batch_decode(outputs)[0]
    except NameError as e:
        print(f"Error in predict_sentiment during model.generate or batch_decode: {e}")
        print("Model or Tokenizer is not defined. Likely due to a failed import.")
        return "Error: Model/Tokenizer not available"


    import re
    match = re.search(r'<\|start_header_id\|>assistant<\|end_header_id\|>(.*?)<\|eot_id\|>', output_text, re.DOTALL)

    if match:
        user_response = match.group(1).strip()
        return user_response
    else:
        print("No user response found.")
        return "unknown"


# Generate Predictions
print(f"Generating predictions from: {TEST_PATH}")
test_data = pd.read_csv(TEST_PATH)
predicted_labels = []

for sentence in test_data['sentence'].tolist():
    label = predict_sentiment(fine_tuned_model, tokenizer, sentence)
    predicted_labels.append(label)

# Submission
print(f"Creating submission from: {SUBMISSION_PATH}")
submission = pd.read_csv(SUBMISSION_PATH)
submission['label'] = predicted_labels
submission.to_csv("submission.csv", index=False)

print("Submission file created.")
print(submission)
print("Sentiment analysis pipeline completed.")

Starting the sentiment analysis pipeline...
Successfully imported Unsloth modules
Loading model: /kaggle/input/llama-3.1/transformers/8b-instruct/2
==((====))==  Unsloth 2025.2.12: Fast Llama patching. Transformers: 4.49.0.
   \\   /|    GPU: Tesla P100-PCIE-16GB. Max memory: 15.888 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 6.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

/kaggle/input/llama-3.1/transformers/8b-instruct/2 does not have a padding token! Will use pad_token = <|finetune_right_pad_id|>.
Applying LoRA...


Unsloth 2025.2.12 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


Loading dataset from: /kaggle/input/multi-lingual-sentiment-analysis/train.csv


...

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Starting training...
GPU = Tesla P100-PCIE-16GB. Max memory = 15.888 GB.
10.873 GB of memory reserved before training.


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1,000 | Num Epochs = 2
O^O/ \_/ \    Batch size per device = 8 | Gradient Accumulation steps = 2
\        /    Total batch size = 16 | Total steps = 80
 "-____-"     Number of trainable parameters = 20,971,520


Step,Training Loss
1,2.798300
2,2.628100
3,2.216600
4,0.812600


OutOfMemoryError: CUDA out of memory. Tried to allocate 2.94 GiB. GPU 0 has a total capacity of 15.89 GiB of which 2.81 GiB is free. Process 2437 has 13.07 GiB memory in use. Of the allocated memory 12.53 GiB is allocated by PyTorch, and 227.01 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)